In [1]:
import mlflow
import pandas as pd
import time
import mlflow.system_metrics
from logging import getLogger, StreamHandler, INFO
import xgboost
import shap
import mlflow
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

/workspace/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mlflow.set_tracking_uri(
    "http://ec2-44-217-145-52.compute-1.amazonaws.com:5000"
)
experiment = mlflow.set_experiment("test")
mlflow.system_metrics.enable_system_metrics_logging()

In [ ]:
run = mlflow.start_run(experiment_id=experiment.experiment_id)

2024/10/27 08:24:42 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


In [4]:
mlflow.log_metrics({"tlearner_auuc": 0.1, "tlearner_uplift": 0.01})
time.sleep(10)
mlflow.log_metrics({"slearner_auuc": 0.2, "tlearner_uplift": 0.02})

In [5]:
# load UCI Adult Data Set; segment it into training and test sets
X, y = shap.datasets.adult()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

# train XGBoost model
model = xgboost.XGBClassifier().fit(X_train, y_train)

# construct an evaluation dataset from the test set
eval_data = X_test
eval_data["target"] = y_test

In [6]:
# load UCI Adult Data Set; segment it into training and test sets
X, y = shap.datasets.adult()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

# train XGBoost model
model = xgboost.XGBClassifier().fit(X_train, y_train)

# construct an evaluation dataset from the test set
eval_data = X_test
eval_data["target"] = y_test

In [7]:

model_info = mlflow.sklearn.log_model(model, "model")
result = mlflow.evaluate(
    model_info.model_uri,
    eval_data,
    targets="target",
    model_type="classifier",
    evaluators=["default"],
)

2024/10/27 08:25:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/workspace/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/10/27 08:25:33 INFO mlflow.models.evaluat

In [8]:
def example_custom_metric_fn(eval_df, builtin_metrics, artifacts_dir):
    """
    This example custom metric function creates a metric based on the ``prediction`` and
    ``target`` columns in ``eval_df`` and a metric derived from existing metrics in
    ``builtin_metrics``. It also generates and saves a scatter plot to ``artifacts_dir`` that
    visualizes the relationship between the predictions and targets for the given model to a
    file as an image artifact.
    """
    metrics = {
        "squared_diff_plus_one": np.sum(
            np.abs(eval_df["prediction"] - eval_df["target"] + 1) ** 2
        ),
        "sum_on_label_divided_by_two": builtin_metrics["sum_on_label"] / 2,
    }
    plt.scatter(eval_df["prediction"], eval_df["target"])
    plt.xlabel("Targets")
    plt.ylabel("Predictions")
    plt.title("Targets vs. Predictions")
    plot_path = os.path.join(artifacts_dir, "example_scatter_plot.png")
    plt.savefig(plot_path)
    artifacts = {"example_scatter_plot_artifact": plot_path}
    return metrics, artifacts

In [9]:
mlflow.log_table(eval_data, artifact_file="eval_data.json")

In [10]:
mlflow.end_run()

2024/10/27 08:25:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run zealous-squid-821 at: http://ec2-44-217-145-52.compute-1.amazonaws.com:5000/#/experiments/1/runs/cd926aa703cd49fbbd291273f85d9ccd.
2024/10/27 08:25:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-44-217-145-52.compute-1.amazonaws.com:5000/#/experiments/1.
2024/10/27 08:25:44 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/10/27 08:25:44 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
